In [1]:
from tqdm import tqdm
import os
import numpy as np
from PIL import Image
import torch

TS = (128, 128)

def Load_Data(directory):
    X = []
    y = []

    class_names = sorted(os.listdir(directory))
    class_to_idx = {cls: i for i, cls in enumerate(class_names)}

    image_paths = []
    labels = []

    for cls in class_names:
        class_path = os.path.join(directory, cls)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            image_paths.append(os.path.join(class_path, img_name))
            labels.append(class_to_idx[cls])

    for image_path, label in tqdm(zip(image_paths, labels),
                                  total=len(image_paths),
                                  desc="Loading images"):
        img = Image.open(image_path).convert("RGB")
        img = img.resize(TS)

        img = np.array(img) / 255.0              # normalize
        img = np.transpose(img, (2, 0, 1))       # HWC → CHW

        X.append(img)
        y.append(label)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    return X, y, class_to_idx


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torch

TS = (128, 128)

class PlantVillageDataset(Dataset):
    def __init__(self, directory):
        self.samples = []
        self.class_names = sorted(os.listdir(directory))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.class_names)}

        for cls in self.class_names:
            cls_path = os.path.join(directory, cls)
            if not os.path.isdir(cls_path):
                continue
            for img_name in os.listdir(cls_path):
                self.samples.append(
                    (os.path.join(cls_path, img_name), self.class_to_idx[cls])
                )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]

        img = Image.open(image_path).convert("RGB")
        img = img.resize(TS)

        img = torch.from_numpy(
            (np.array(img) / 255.0).transpose(2, 0, 1)
        ).float()

        return img, torch.tensor(label)

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
X, y, class_to_idx = Load_Data("plantvillage/color")
X = X.to(device)
y = y.to(device)

Loading images: 100%|████████████████████████████████████████████████████████████| 54305/54305 [35:42<00:00, 25.35it/s]


RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 10676797440 bytes.

In [4]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)

train_loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

Loading images: 100%|████████████████████████████████████████████████████████████| 54305/54305 [25:57<00:00, 34.86it/s]


Dummy Accuracy: 0.101648098701777


In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),                  # converts to [0,1]
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [7]:
from torch.utils.data import random_split
data_dir = "plantvillage/color"

full_dataset = datasets.ImageFolder(
    root=data_dir,
    transform=train_transform
)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform


In [8]:
BATCH_SIZE = 32   # reduce to 16 if RAM is low

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [13]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 56 * 56, 128),  # 224 → 112 → 56
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [15]:
num_classes = len(full_dataset.classes)

model = SimpleCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
from tqdm.notebook import tqdm

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(
        train_loader,
        desc=f"Epoch [{epoch+1}/{EPOCHS}]",
        leave=True
    )

    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        progress_bar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{100*correct/total:.2f}%"
        })

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Acc: {train_acc:.2f}%")


Epoch [1/10]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch [1/10] | Train Acc: 99.45%


Epoch [2/10]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch [2/10] | Train Acc: 99.44%


Epoch [3/10]:   0%|          | 0/1358 [00:00<?, ?it/s]

...

Epoch [8/10] | Train Acc: 99.44%


Epoch [9/10]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch [9/10] | Train Acc: 99.52%


Epoch [10/10]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch [10/10] | Train Acc: 99.61%


In [20]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 85.79%


In [9]:
import torchvision.models as models
num_classes = len(full_dataset.classes)

model = models.mobilenet_v2(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.last_channel, num_classes)
)

C:\Users\mastr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\mastr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\mastr/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|█████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:02<00:00, 6.67MB/s]


In [10]:
model = model.to(device)

In [11]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    model.classifier.parameters(),  # ONLY head
    lr=1e-3
)

In [12]:
from tqdm.notebook import tqdm

EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    correct, total, running_loss = 0, 0, 0

    pbar = tqdm(train_loader, desc=f"TL Epoch [{epoch+1}/{EPOCHS}]", leave=True)

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "acc": f"{100*correct/total:.2f}%"
        })

    print(f"Epoch {epoch+1} Train Acc: {100*correct/total:.2f}%")

TL Epoch [1/5]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch 1 Train Acc: 89.81%


TL Epoch [2/5]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch 2 Train Acc: 94.71%


TL Epoch [3/5]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch 3 Train Acc: 95.43%


TL Epoch [4/5]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch 4 Train Acc: 95.80%


TL Epoch [5/5]:   0%|          | 0/1358 [00:00<?, ?it/s]

Epoch 5 Train Acc: 95.92%
